In [1]:
import pandas as pd
import json

In [2]:
def colate(file_path):
    data = pd.read_csv(file_path)
    df = pd.DataFrame(data)
    sites = df['Site Num'].value_counts()
    data_object={}
    for i in sites.index:
        data_object[i]={}
        hold = df.loc[df['Site Num']==i]
        hold.reset_index(inplace=True)
        data_object[i]['site_info']={}
        data_object[i]['site_info']['fips']=str(hold['State Code'][0]*1000+hold['County Code'][0])
        data_object[i]['site_info']['State']=hold['State Name'][0]
        data_object[i]['site_info']['County']=hold['County Name'][0]
        data_object[i]['site_info']['lat']=hold['Latitude'][0]
        data_object[i]['site_info']['lon']=hold['Longitude'][0]
        data_object[i]['data']={}
        for x,y in hold.iterrows():
            data_object[i]['data'][y['Date Local']]= y['Arithmetic Mean']
    return data_object

In [3]:
pollutant_data = {}
ozone_data = colate('rawdata/daily_44201_2020.csv')
so2_data=colate('rawdata/daily_42401_2020.csv')
no2_data=colate('rawdata/daily_42602_2020.csv')
co_data = colate('rawdata/daily_42101_2020.csv')
pm_data = colate('rawdata/daily_88101_2020.csv')

In [4]:
def append_pol(dath, pol):
    for i in dath: 
        fips_hold = dath[i]['site_info']['fips']
        if fips_hold in  pollutant_data:
            if pol in pollutant_data[fips_hold]:
                pollutant_data[fips_hold][pol][i]={}
                pollutant_data[fips_hold][pol][i]['data']=dath[i]['data']
                pollutant_data[fips_hold][pol][i]['site_coord']={}
                pollutant_data[fips_hold][pol][i]['site_coord']['lon']=dath[i]['site_info']['lon']
                pollutant_data[fips_hold][pol][i]['site_coord']['lat']=dath[i]['site_info']['lat']
            else:
                pollutant_data[fips_hold][pol]={}
                pollutant_data[fips_hold][pol][i]={}
                pollutant_data[fips_hold][pol][i]['data']=dath[i]['data']
                pollutant_data[fips_hold][pol][i]['site_coord']={}
                pollutant_data[fips_hold][pol][i]['site_coord']['lon']=dath[i]['site_info']['lon']
                pollutant_data[fips_hold][pol][i]['site_coord']['lat']=dath[i]['site_info']['lat']
        else:
            pollutant_data[fips_hold] ={}
            pollutant_data[fips_hold]['fips'] = fips_hold
            pollutant_data[fips_hold]['info']={}
            pollutant_data[fips_hold]['info']['State']=dath[i]['site_info']['State']
            pollutant_data[fips_hold]['info']['County']=dath[i]['site_info']['County']
            pollutant_data[fips_hold][pol]={}
            pollutant_data[fips_hold][pol][i]={}
            pollutant_data[fips_hold][pol][i]['data']=dath[i]['data']
            pollutant_data[fips_hold][pol][i]['site_coord']={}
            pollutant_data[fips_hold][pol][i]['site_coord']['lon']=dath[i]['site_info']['lon']
            pollutant_data[fips_hold][pol][i]['site_coord']['lat']=dath[i]['site_info']['lat']

In [5]:
pollutant_data={}
append_pol(ozone_data,'ozone')
append_pol(so2_data, 'SO2')
append_pol(no2_data, 'NO2')
append_pol(co_data,'CO')
append_pol(pm_data,'PM_25')

In [6]:
with open('data/pol_dat2.json', 'w') as outfile:
    json.dump(pollutant_data, outfile)

In [7]:
pollutant_json=json.dumps(pollutant_data)
parsed = json.loads(pollutant_json)
print(json.dumps(parsed, indent=4, sort_keys=True))

{
    "10003": {
        "PM_25": {
            "1007": {
                "data": {
                    "2020-01-01": 6.2,
                    "2020-01-02": 4.7,
                    "2020-01-03": 3.8,
                    "2020-01-04": 4.7,
                    "2020-01-05": 5.5,
                    "2020-01-06": 3.4,
                    "2020-01-07": 2.7,
                    "2020-01-08": 5.8,
                    "2020-01-09": 9.3,
                    "2020-01-10": 4.1,
                    "2020-01-11": 5.5,
                    "2020-01-12": 5.2,
                    "2020-01-13": 8.4,
                    "2020-01-14": 7.3,
                    "2020-01-15": 4.0,
                    "2020-01-16": 9.1,
                    "2020-01-17": 8.5,
                    "2020-01-18": 3.1,
                    "2020-01-19": 7.9,
                    "2020-01-20": 10.0,
                    "2020-01-21": 5.5,
                    "2020-01-22": 4.4,
                    "2020-01-23": 3.2,
                  

In [14]:
data = pd.read_csv('rawdata/daily_aqi_by_county_2020.csv')
df = pd.DataFrame(data)
df['fips']=df['State Code']*1000+df['County Code']
sites = df['fips'].value_counts()
aqi_data={}
for i in sites.index:
    aqi_data[i]={}
    hold = df.loc[df['fips']==i]
    hold.reset_index(inplace=True)
    aqi_data[i]['fips'] = i
    aqi_data[i]['info']={}
    aqi_data[i]['info']['State']=hold['State Name'][0]
    aqi_data[i]['info']['County']=hold['county Name'][0]
    aqi_data[i]['data']={}
    for x,y in hold.iterrows():
        aqi_data[i]['data'][y['Date']]= y['AQI']

In [ ]:
geo = json.load(open("gz_2010_us_050_00_5m.json"))

In [9]:
with open('data/aqi_dat2.json', 'w') as outfile:
    json.dump(aqi_data, outfile)

In [35]:
for i in geo['features']:
    fips=int(i['properties']['STATE'])*1000+int(i['properties']['COUNTY'])
    count=0
    aqisum=0
    exists = False
    try:
        G=aqi_data[fips]['data']
        for key in G:
            count += 1
            aqisum += G[key]
            aqi_avg = aqisum/count
            print(aqi_avg)
            i['properties']['aqi_avg']=[aqi_avg]
    except:
        print(f'No {fips}')
        i['properties']['aqi_avg']= 'NA' 

No 2261
No 2270
No 2275
No 2290
39.0
37.5
37.666666666666664
37.25
37.4
38.166666666666664
38.42857142857143
39.5
40.333333333333336
40.2
40.18181818181818
40.333333333333336
40.38461538461539
40.92857142857143
41.93333333333333
41.375
41.11764705882353
40.833333333333336
40.8421052631579
40.75
40.23809523809524
39.95454545454545
39.73913043478261
39.666666666666664
39.68
39.65384615384615
39.77777777777778
39.857142857142854
39.96551724137931
40.06666666666667
40.16129032258065
40.125
40.121212121212125
40.088235294117645
40.114285714285714
40.083333333333336
40.027027027027025
40.0
40.05128205128205
40.05
39.58536585365854
39.61904761904762
39.627906976744185
39.63636363636363
39.733333333333334
39.80434782608695
39.93617021276596
40.020833333333336
40.08163265306123
40.08
40.13725490196079
39.92307692307692
39.716981132075475
39.75925925925926
39.74545454545454
39.857142857142854
39.87719298245614
39.87931034482759
39.88135593220339
39.916666666666664
39.52459016393443
39.2096774193

27.527272727272727
27.464285714285715
27.54385964912281
27.70689655172414
27.864406779661017
28.05
27.62295081967213
No 21073
No 21081
No 21083
No 21097
No 21103
No 21113
No 21129
No 21137
No 21141
No 21149
No 21157
No 21165
No 21169
No 21179
No 21189
No 21195
No 21201
No 21215
11.0
23.0
21.0
22.75
22.2
23.833333333333332
23.714285714285715
25.25
26.666666666666668
30.1
31.727272727272727
No 21237
No 22003
No 22013
No 22027
No 22037
30.0
30.5
30.666666666666668
31.5
33.2
34.666666666666664
35.285714285714285
35.625
36.333333333333336
35.8
35.54545454545455
33.916666666666664
31.846153846153847
31.214285714285715
30.866666666666667
30.4375
30.41176470588235
30.72222222222222
31.05263157894737
31.25
31.428571428571427
31.181818181818183
31.217391304347824
31.541666666666668
31.8
31.807692307692307
31.85185185185185
31.928571428571427
31.96551724137931
32.1
32.064516129032256
32.1875
32.666666666666664
33.029411764705884
33.142857142857146
33.083333333333336
32.945945945945944
33.23684210

32.87179487179487
32.81012658227848
32.825
32.67901234567901
32.68292682926829
32.80722891566265
33.17857142857143
33.529411764705884
33.86046511627907
34.01149425287356
34.15909090909091
34.235955056179776
34.36666666666667
32.0
31.5
29.0
30.25
30.6
30.666666666666668
30.571428571428573
32.0
32.333333333333336
33.1
33.36363636363637
33.333333333333336
34.84615384615385
35.785714285714285
36.93333333333333
37.6875
36.8235294117647
36.666666666666664
36.36842105263158
36.0
35.523809523809526
35.04545454545455
34.82608695652174
34.583333333333336
34.68
34.61538461538461
34.851851851851855
34.535714285714285
34.51724137931034
34.46666666666667
34.41935483870968
34.3125
34.303030303030305
34.205882352941174
34.457142857142856
34.361111111111114
34.45945945945946
34.26315789473684
34.333333333333336
34.3
34.390243902439025
34.30952380952381
34.2093023255814
34.15909090909091
34.08888888888889
34.19565217391305
34.744680851063826
35.1875
35.10204081632653
35.14
35.15686274509804
35.173076923

29.724137931034484
29.733333333333334
29.70967741935484
29.65625
29.87878787878788
29.970588235294116
30.228571428571428
30.27777777777778
30.16216216216216
30.07894736842105
30.128205128205128
30.125
30.195121951219512
30.023809523809526
30.209302325581394
30.34090909090909
30.533333333333335
30.73913043478261
30.93617021276596
31.0625
31.20408163265306
31.28
31.431372549019606
31.634615384615383
31.62264150943396
31.61111111111111
31.763636363636362
31.910714285714285
32.0
32.05172413793103
32.101694915254235
32.266666666666666
32.40983606557377
32.45161290322581
32.476190476190474
32.5625
32.676923076923075
32.75757575757576
32.92537313432836
33.05882352941177
33.15942028985507
33.2
33.19718309859155
33.25
33.273972602739725
33.36486486486486
33.45333333333333
33.5921052631579
33.688311688311686
33.73076923076923
33.822784810126585
33.9375
34.06172839506173
34.18292682926829
34.265060240963855
34.392857142857146
34.55294117647059
34.69767441860465
34.735632183908045
34.8181818181818

34.723684210526315
35.116883116883116
35.5
35.78481012658228
35.9
35.74074074074074
35.73170731707317
35.602409638554214
35.5
35.32941176470588
35.36046511627907
35.62068965517241
35.84090909090909
35.95505617977528
35.86666666666667
35.86813186813187
No 13091
47.0
39.5
39.0
33.5
31.6
32.0
32.142857142857146
35.375
39.0
39.9
39.18181818181818
38.416666666666664
38.23076923076923
37.5
36.86666666666667
36.25
35.76470588235294
35.611111111111114
34.78947368421053
34.4
35.19047619047619
35.04545454545455
35.91304347826087
35.875
35.76
35.53846153846154
35.2962962962963
35.357142857142854
36.06896551724138
35.833333333333336
35.38709677419355
35.25
34.878787878787875
35.529411764705884
35.68571428571428
35.5
35.4054054054054
35.026315789473685
34.92307692307692
35.05
34.926829268292686
34.976190476190474
34.906976744186046
34.61363636363637
34.6
34.47826086956522
34.191489361702125
34.104166666666664
33.55102040816327
33.4
33.294117647058826
33.65384615384615
33.9622641509434
33.7037037037

...

15.5
14.333333333333334
18.75
18.2
No 21023
No 21025
No 21027
No 21031
No 21033
13.0
23.5
25.0
24.75
23.8
25.166666666666668
26.714285714285715
28.125
27.77777777777778
29.7
31.636363636363637
No 21039
No 20007
No 20011
No 20015
No 20019
No 20021
No 20023
No 20035
No 20037
No 20039
No 20043
No 20045
No 20051
No 20053
No 20057
No 20059
No 20061
No 20065
No 20069
No 20071
No 20075
No 29055
No 29059
No 29061
No 29065
No 29069
No 29071
No 29073
No 29077
No 29081
No 29083
No 29087
No 29089
No 29091
No 29097
No 29099
35.0
34.5
34.0
34.0
33.0
32.666666666666664
33.857142857142854
34.0
36.111111111111114
36.3
36.36363636363637
35.5
34.84615384615385
34.285714285714285
34.733333333333334
34.8125
34.8235294117647
34.77777777777778
35.31578947368421
35.1
35.19047619047619
35.22727272727273
35.0
34.958333333333336
35.28
35.46153846153846
35.44444444444444
35.392857142857146
35.48275862068966
35.36666666666667
35.16129032258065
34.8125
34.72727272727273
34.94117647058823
35.2
35.19444444444444
35.3

2.7083333333333335
2.958904109589041
2.918918918918919
2.88
2.8421052631578947
2.857142857142857
2.8205128205128207
2.7974683544303796
2.9
2.9753086419753085
2.9390243902439024
2.9036144578313254
2.869047619047619
2.835294117647059
2.813953488372093
2.8620689655172415
2.840909090909091
3.0
2.966666666666667
2.934065934065934
No 37015
0.0
0.0
0.0
0.0
0.0
0.5
0.42857142857142855
0.375
4.666666666666667
4.2
3.8181818181818183
3.5
3.230769230769231
3.0
3.4
3.1875
3.764705882352941
5.777777777777778
5.473684210526316
5.2
4.9523809523809526
4.7727272727272725
5.0
5.25
5.04
4.846153846153846
4.666666666666667
4.5
4.827586206896552
4.666666666666667
4.806451612903226
4.65625
4.515151515151516
4.382352941176471
4.285714285714286
4.166666666666667
4.054054054054054
3.9473684210526314
4.205128205128205
4.275
4.170731707317073
4.071428571428571
4.3023255813953485
4.204545454545454
4.111111111111111
5.369565217391305
6.042553191489362
6.479166666666667
6.36734693877551
6.24
6.117647058823529
6.0
7.

1.2826086956521738
1.3191489361702127
1.3541666666666667
1.346938775510204
1.36
1.392156862745098
1.4615384615384615
1.528301886792453
1.5555555555555556
1.5636363636363637
1.5535714285714286
1.543859649122807
1.5689655172413792
1.5932203389830508
1.6666666666666667
1.721311475409836
1.7258064516129032
1.6984126984126984
1.6875
1.7076923076923076
1.7272727272727273
1.7761194029850746
1.8235294117647058
1.826086956521739
1.8
1.7887323943661972
1.7777777777777777
1.7534246575342465
1.7432432432432432
1.7466666666666666
1.763157894736842
1.7532467532467533
1.7435897435897436
1.7215189873417722
1.7
1.7037037037037037
1.7195121951219512
1.7108433734939759
1.6904761904761905
1.6705882352941177
1.6511627906976745
1.632183908045977
1.6136363636363635
1.595505617977528
1.5777777777777777
1.5604395604395604
1.5434782608695652
1.5376344086021505
1.5212765957446808
1.5052631578947369
1.4895833333333333
1.4845360824742269
1.4795918367346939
1.4646464646464648
1.46
1.4554455445544554
1.4509803921568

9.375
9.666666666666666
9.3
10.272727272727273
9.916666666666666
9.846153846153847
10.285714285714286
10.4
11.5
13.058823529411764
15.38888888888889
17.36842105263158
19.2
19.38095238095238
18.772727272727273
18.608695652173914
18.375
18.56
19.153846153846153
18.703703703703702
18.321428571428573
17.896551724137932
18.1
17.741935483870968
44.0
33.5
44.0
58.25
59.0
50.666666666666664
59.714285714285715
59.75
54.44444444444444
49.3
46.36363636363637
45.833333333333336
42.61538461538461
40.07142857142857
37.733333333333334
36.4375
35.23529411764706
33.333333333333336
32.10526315789474
31.35
30.952380952380953
31.40909090909091
30.73913043478261
32.0
31.2
32.34615384615385
33.888888888888886
32.964285714285715
32.13793103448276
31.4
32.70967741935484
7.0
7.5
7.333333333333333
8.0
8.0
7.5
7.142857142857143
7.125
7.888888888888889
8.1
8.090909090909092
7.833333333333333
7.923076923076923
9.0
9.2
9.8125
10.647058823529411
10.555555555555555
10.473684210526315
10.7
10.714285714285714
10.5
10.3

5.590909090909091
5.478260869565218
5.375
5.24
5.1923076923076925
5.111111111111111
5.035714285714286
5.0344827586206895
4.966666666666667
4.967741935483871
4.90625
4.909090909090909
6.029411764705882
6.3428571428571425
6.638888888888889
6.54054054054054
6.447368421052632
6.333333333333333
6.275
6.219512195121951
6.166666666666667
6.093023255813954
6.090909090909091
6.088888888888889
6.065217391304348
6.042553191489362
6.020833333333333
6.020408163265306
5.94
5.862745098039215
5.826923076923077
5.7924528301886795
5.888888888888889
5.909090909090909
5.928571428571429
5.9298245614035086
5.931034482758621
6.0508474576271185
6.1
6.081967213114754
6.096774193548387
6.079365079365079
6.109375
6.138461538461539
6.181818181818182
6.268656716417911
6.3088235294117645
6.318840579710145
6.314285714285714
6.338028169014085
6.361111111111111
6.3561643835616435
6.351351351351352
6.333333333333333
6.315789473684211
6.311688311688312
6.3076923076923075
6.3544303797468356
6.35
6.345679012345679
6.37804

In [36]:
with open('data/geojson_withNAs.json', 'w') as outfile:
    json.dump(geo, outfile)